In [1]:
import os
# import streamlit as st
import pickle
import time
import langchain
# from langchain import OpenAI
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQAWithSourcesChain, RetrievalQA
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
# from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

In [2]:
load_dotenv(dotenv_path="../.env")

True

In [3]:
# help(OpenAI)

In [4]:
# llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9, max_tokens=500, api_key=os.getenv("OPENAI_API_KEY")) # Default model is "gpt-3.5-turbo-instruct"

# loaders = UnstructuredURLLoader(urls=[
#     'https://www.pharmacytimes.com/view/kraken-trial-highlights-muvalaplin-s-efficacy-safety-in-reducing-lipoprotein-a-',
#     'https://link.springer.com/article/10.1007/s11096-023-01658-x'
# ])

# data = loaders.load()
# len(data)

In [5]:
# llm.model_name

In [6]:
# llm.invoke("What is one piece?")

In [7]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.9,
    max_tokens=500,
    groq_api_key=os.getenv("GROQ_API_KEY")
    
)
response=llm.invoke("What is Hunter x Hunter?")
print(response.content)

Hunter x Hunter is a popular Japanese manga and anime series created by Yoshihiro Togashi. The story follows the adventures of Gon Freecss, a young boy who aspires to become a Hunter, a prestigious title granted to individuals who have proven their courage and abilities.

In the world of Hunter x Hunter, Hunters are licensed professionals who have demonstrated exceptional skills and accomplishments in various fields, such as exploration, research, and combat. They are highly respected and admired, and their licenses grant them access to exclusive opportunities and resources.

Gon, the main protagonist, sets out on a journey to become a Hunter, accompanied by his friends Killua, Kurapika, and Leorio. Along the way, they face numerous challenges, battle fierce enemies, and uncover dark secrets about the world they live in.

The series is known for its unique blend of action, adventure, fantasy, and comedy, as well as its complex characters, intricate plotlines, and themes that explore fr

In [8]:
llm.max_tokens

500

In [9]:
# help(RecursiveCharacterTextSplitter)

In [10]:
loaders = UnstructuredURLLoader(urls=[
    'https://www.pharmacytimes.com/view/kraken-trial-highlights-muvalaplin-s-efficacy-safety-in-reducing-lipoprotein-a-',
    'https://link.springer.com/article/10.1007/s11096-023-01658-x'
])

data = loaders.load()
len(data)

2

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

docs = text_splitter.split_documents(data)
len(docs)

72

In [12]:
docs[5]

Document(metadata={'source': 'https://www.pharmacytimes.com/view/kraken-trial-highlights-muvalaplin-s-efficacy-safety-in-reducing-lipoprotein-a-'}, page_content="Author(s):\n\nAlexandra Gerlach, Associate Editor\n\nKey Takeaways\n\nMuvalaplin significantly reduces Lp(a) levels, with up to 85.8% reduction at the highest dose in the KRAKEN trial.\n\nThe trial showed dose-dependent decreases in apolipoprotein B levels, with no significant changes in high-sensitivity C-reactive protein.\n\nNo safety or tolerability concerns were identified, suggesting muvalaplin's potential as a therapeutic option for Lp(a) reduction.\n\nFurther studies are needed to confirm long-term efficacy and impact on cardiovascular outcomes.\n\nSHOW MORE\n\nMuvalaplin is an oral, small molecule inhibitor that can reduce lipoprotein(a).\n\nMuvalaplin (LY3473329; Eli Lilly and Company) demonstrated meaningful improvements in reducing lipoprotein(a) (Lp[a]) and was well tolerated by patients with Lp(a) at high risk of 

In [13]:
# help(OpenAIEmbeddings)

In [14]:
# embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY")) # Default embedding model is "text-embedding-ada-002"

# vectorindex_openai = FAISS.from_documents(documents=docs, embedding=embeddings)
# embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=os.getenv("OPENAI_API_KEY"))
# embeddings.model

In [15]:
embeddings = SentenceTransformerEmbeddings(model_name="avsolatorio/GIST-small-Embedding-v0")

vectorindex = FAISS.from_documents(documents=docs, embedding=embeddings)

C:\Users\micha\AppData\Local\Temp\ipykernel_39020\3507564277.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="avsolatorio/GIST-small-Embedding-v0")
c:\Users\micha\Anaconda3\envs\LLMEnv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
embeddings.model_name

'avsolatorio/GIST-small-Embedding-v0'

In [17]:
file_path = "vector_index.pkl"
with open(file=file_path, mode='wb') as f:
    pickle.dump(vectorindex, f)

In [18]:
if os.path.exists(path=file_path):
    with open(file=file_path, mode='rb') as f:
        vectorIndex = pickle.load(f)

In [19]:
help(vectorIndex.as_retriever)

Help on method as_retriever in module langchain_core.vectorstores.base:

as_retriever(**kwargs: 'Any') -> 'VectorStoreRetriever' method of langchain_community.vectorstores.faiss.FAISS instance
    Return VectorStoreRetriever initialized from this VectorStore.
    
    Args:
        **kwargs: Keyword arguments to pass to the search function.
            Can include:
            search_type (Optional[str]): Defines the type of search that
                the Retriever should perform.
                Can be "similarity" (default), "mmr", or
                "similarity_score_threshold".
            search_kwargs (Optional[Dict]): Keyword arguments to pass to the
                search function. Can include things like:
                    k: Amount of documents to return (Default: 4)
                    score_threshold: Minimum relevance threshold
                        for similarity_score_threshold
                    fetch_k: Amount of documents to pass to MMR algorithm
               

In [20]:
retriever = vectorIndex.as_retriever(
    search_kwargs={"k": 5}
    )

In [21]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=retriever)
print(chain)

verbose=False combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021E39D45660>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021E39D46230>, model_name='llama-3.3-70b-versatile', temperature=0.9, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=500), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summ

In [22]:
type(chain)

langchain.chains.qa_with_sources.retrieval.RetrievalQAWithSourcesChain

In [23]:
# help(chain)

In [ ]:
query = "What is the aim of Scope, content and quality of clinical pharmacy practice guidelines paper?"

langchain.debug = False

chain({"question": query}, return_only_outputs=True)

In [ ]:
langchain.debug = True
chain({"question": query}, return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What is the aim of Scope, content and quality of clinical pharmacy practice guidelines paper?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Scope, content and quality of clinical pharmacy practice guidelines: a systematic review\n\nDownload PDF\n\nVibhu Paudyal ORCID: orcid.org/0000-0002-4173-64901,\n\nBetul Okuyan ORCID: orcid.org/0000-0002-4023-25652,\n\nMartin Charles Henman3,\n\nDerek Stewart ORCID: orcid.org/0000-0001-7360-85924,\n\nDaniela Fialová5,6,\n\nAnkie Hazen7,\n\nMonika Lutters8,\n\nAnna Oleárová9,\n\nAnita E. Weidmann10,\n\nFrancesca Wirth11,\n\nCathal A. Cadogan12 &\n\n…\n\nZachariah Nazar ORCID: orcid.org/0000-0003-4104-4221

{'answer': 'The aim of the "Scope, content and quality of clinical pharmacy practice guidelines" paper is to review the scope of clinical pharmacy guidelines and assess the extent to which these guidelines conform to quality standards as per the AGREE II instrument.\n\n',
 'sources': 'https://link.springer.com/article/10.1007/s11096-023-01658-x'}

In [32]:
## FAISS GPU
import faiss
res = faiss.StandardGpuResources()

if os.path.exists(path=file_path):
    with open(file=file_path, mode='rb') as f:
        vectorIndex = pickle.load(f)
        
gpu_index = faiss.index_cpu_to_gpu(provider=res, device=0, index=vectorIndex.index)
vectorIndex.index = gpu_index

In [33]:
retriever = vectorIndex.as_retriever(search_kwargs={"k": 5})

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=retriever) # Default -> MapReduceDocumentsChain, The output consists of question, answers and sources
chain({"question": query}, return_only_outputs=True) # return_only_outputs=True -> Only returns the answer and its sources

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What is the aim of Scope, content and quality of clinical pharmacy practice guidelines paper?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Scope, content and quality of clinical pharmacy practice guidelines: a systematic review\n\nDownload PDF\n\nVibhu Paudyal ORCID: orcid.org/0000-0002-4173-64901,\n\nBetul Okuyan ORCID: orcid.org/0000-0002-4023-25652,\n\nMartin Charles Henman3,\n\nDerek Stewart ORCID: orcid.org/0000-0001-7360-85924,\n\nDaniela Fialová5,6,\n\nAnkie Hazen7,\n\nMonika Lutters8,\n\nAnna Oleárová9,\n\nAnita E. Weidmann10,\n\nFrancesca Wirth11,\n\nCathal A. Cadogan12 &\n\n…\n\nZachariah Nazar ORCID: orcid.org/0000-0003-4104-4221

{'answer': 'The aim of the "Scope, content and quality of clinical pharmacy practice guidelines" paper is to review the scope of clinical pharmacy guidelines and assess the extent to which these guidelines conform to quality standards as per the AGREE II instrument.\n',
 'sources': 'https://link.springer.com/article/10.1007/s11096-023-01658-x'}

In [ ]:
from langchain.chains import RetrievalQA 

chain = RetrievalQA.from_llm(llm=llm, retriever=retriever) # Default -> StuffDocumentsChain, Without sources
chain({"query": query}, return_only_outputs=True) # return_only_outputs=True -> Only returns the answer

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is the aim of Scope, content and quality of clinical pharmacy practice guidelines paper?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "What is the aim of Scope, content and quality of clinical pharmacy practice guidelines paper?",
  "context": "Context:\nScope, content and quality of clinical pharmacy practice guidelines: a systematic review\n\nDownload PDF\n\nVibhu Paudyal ORCID: orcid.org/0000-0002-4173-64901,\n\nBetul Okuyan ORCID: orcid.org/0000-0002-4023-25652,\n\nMartin Charles Henman3,\n\nDerek Stewart ORCID: orcid.org/0000-0001-7360-85924,\n\nDaniela Fialová5,6,\n\nAnkie Hazen7,\n\nMonika Lutters8,\n\nAnna Oleárová9,\n\nAnita E. Weidmann10,\n\nFrancesca Wirth11,\n\nCathal A. Cadogan12 &\n\n…\n\nZachariah Nazar OR

{'result': 'The aim of the "Scope, content and quality of clinical pharmacy practice guidelines" paper is to review the scope of clinical pharmacy guidelines and assess the extent to which these guidelines conform to quality standards as per the AGREE II instrument.'}

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_chain_type(chain_type="stuff", llm=llm, retriever=retriever) # Assigning chain_type="stuff" -> StuffDocumentsChain
chain({"question": query}, return_only_outputs=True) # return_only_outputs=True -> Only returns the answer and its sources

In [ ]:
from langchain.chains import RetrievalQA 

chain = RetrievalQA.from_chain_type(chain_type="map_reduce", llm=llm, retriever=retriever) # Assigning chain_type="map_reduce" -> MapReduceDocumentsChain
chain({"query": query}, return_only_outputs=True) # return_only_outputs=True -> Only returns the answer

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is the aim of Scope, content and quality of clinical pharmacy practice guidelines paper?"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Scope, content and quality of clinical pharmacy practice guidelines: a systematic review\n\nDownload PDF\n\nVibhu Paudyal ORCID: orcid.org/0000-0002-4173-64901,\n\nBetul Okuyan ORCID: orcid.org/0000-0002-4023-25652,\n\nMartin Charles Henman3,\n\nDerek Stewart ORCID: orcid.org/0000-0001-7360-85924,\n\nDaniela Fialová5,6,\n\nAnkie Hazen7,\n\nMonika Lutters8,\n\nAnna Oleárová9,\n\nAnita E. Weidmann10,\n\nFrancesca Wirth11,\n\nCathal A. Cadogan12 &\n\n…\n\nZachariah Nazar ORCID: orcid.org/0000-0003-4104-42214\n\n4159 Accesses\n\n9 Citations\n\n6 Altmetric\n\

{'result': 'The aim of the "Scope, content and quality of clinical pharmacy practice guidelines" paper is to review the scope of clinical pharmacy guidelines and assess the extent to which these guidelines conform to quality standards, using the AGREE II instrument. In other words, the aim is to review and evaluate the scope, content, and quality of clinical pharmacy practice guidelines through a systematic review.'}